# Classification Model for Bearing Fault Detection

In [1]:
# Importing the required libraries.
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import r2_score

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV,KFold
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestRegressor

from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import xgboost
from xgboost import XGBRegressor,XGBClassifier

In [2]:
mdf=pd.read_csv('paderborn_features.csv')
mdf

,Min,Max,Mean,Rms,Var,Std,Power,Peak,P2p,CrestFactor,Skew,Kurtosis,Max_f,Sum_f,Mean_f,Var_f,Peak_f,Skew_f,Kurtosis_f,Fault
0,-2.657320,2.565023,0.019567,1.653708,2.734366,1.653592,2.734749,2.657320,5.222343,1.606886,0.011772,-1.492651,5440.108717,10974.54621,2.734749,14742.11586,5440.108717,0.011772,-1.492651,0
1,-2.589131,2.650432,0.016397,1.654484,2.737049,1.654403,2.737318,2.650432,5.239563,1.601969,0.006814,-1.488222,5443.924726,10984.85529,2.737318,14762.82527,5443.924726,0.006814,-1.488222,0
2,-2.711734,2.596707,0.023189,1.652213,2.729270,1.652050,2.729808,2.711734,5.308441,1.641274,0.005670,-1.488567,5428.473371,10954.71838,2.729808,14679.12445,5428.473371,0.005670,-1.488567,0
3,-2.691759,2.651121,0.026191,1.653913,2.734743,1.653706,2.735429,2.691759,5.342880,1.627509,0.006561,-1.492452,5449.700249,10977.27710,2.735429,14794.11094,5449.700249,0.006561,-1.492452,0
4,-2.554691,2.637345,0.018743,1.658265,2.749492,1.658159,2.749843,2.637345,5.192037,1.590424,0.004608,-1.497915,5478.699621,11035.12111,2.749843,14951.98068,5478.699621,0.004608,-1.497915,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21755,-2.866710,2.642856,-0.083886,1.816876,3.294002,1.814939,3.301039,2.866710,5.509565,1.577823,-0.004678,-1.504588,6577.357954,13204.15601,3.301039,21620.12451,6577.357954,-0.004678,-1.504588,2
21756,-2.950052,2.762015,-0.077491,1.817828,3.298493,1.816175,3.304497,2.950052,5.712067,1.622845,-0.006128,-1.506578,6587.620925,13217.98952,3.304497,21687.60266,6587.620925,-0.006128,-1.506578,2
21757,-2.852245,2.626325,-0.075480,1.820374,3.308063,1.818808,3.313761,2.852245,5.478570,1.566846,-0.007347,-1.506746,6599.067412,13255.04265,3.313761,21763.00396,6599.067412,-0.007347,-1.506746,2
21758,-2.888062,2.912858,-0.078450,1.821039,3.310028,1.819348,3.316182,2.912858,5.800920,1.599558,-0.004223,-1.505193,6595.023243,13264.72974,3.316182,21736.35028,6595.023243,-0.004223,-1.505193,2


In [3]:
mdf_train,mdf_test=train_test_split(mdf,train_size=0.8,random_state=100)
mdf_train

,Min,Max,Mean,Rms,Var,Std,Power,Peak,P2p,CrestFactor,Skew,Kurtosis,Max_f,Sum_f,Mean_f,Var_f,Peak_f,Skew_f,Kurtosis_f,Fault
1000,-2.497523,2.870154,0.042149,1.683739,2.833201,1.683212,2.834978,2.870154,5.367676,1.704631,0.012376,-1.493275,5658.805750,11339.911320,2.834978,16003.020020,5658.805750,0.012376,-1.493275,0
13101,-2.954185,2.763392,-0.077159,1.802057,3.241454,1.800404,3.247408,2.954185,5.717577,1.639341,-0.013933,-1.493596,6473.716047,12989.630920,3.247408,20944.099510,6473.716047,-0.013933,-1.493596,1
9700,-1.686138,1.735730,-0.050669,0.943014,0.886708,0.941652,0.889275,1.735730,3.421868,1.840620,0.010348,-1.484479,1765.075491,3557.101086,0.889275,1556.984039,1765.075491,0.010348,-1.484479,1
10567,-1.686138,1.533917,-0.020761,0.926004,0.857052,0.925771,0.857483,1.686138,3.220055,1.820876,-0.007855,-1.488106,1701.654204,3429.931470,0.857483,1447.082788,1701.654204,-0.007855,-1.488106,1
7747,-2.899771,2.539538,0.030609,1.689492,2.853448,1.689215,2.854385,2.899771,5.439309,1.716356,0.000974,-1.492002,5696.899866,11417.539210,2.854385,16219.193330,5696.899866,0.000974,-1.492002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16304,-1.746062,1.478814,-0.011733,0.894588,0.800150,0.894511,0.800288,1.746062,3.224876,1.951805,-0.009773,-1.481636,1589.240029,3201.951084,0.800288,1261.888882,1589.240029,-0.009773,-1.481636,2
79,-2.621503,2.731708,0.024837,1.708605,2.918714,1.708425,2.919331,2.731708,5.353212,1.598795,0.007260,-1.494888,5827.962917,11677.324770,2.919331,16974.057820,5827.962917,0.007260,-1.494888,0
12119,-2.848112,2.601529,-0.031598,1.716623,2.945796,1.716332,2.946795,2.848112,5.449641,1.659137,-0.004027,-1.505798,5882.912171,11787.178630,2.946795,17295.651410,5882.912171,-0.004027,-1.505798,1
14147,-2.938343,2.717244,-0.045351,1.725585,2.975588,1.724989,2.977645,2.938343,5.655587,1.702809,0.002170,-1.494338,5941.506881,11910.579430,2.977645,17641.904970,5941.506881,0.002170,-1.494338,2


In [4]:
y_train=mdf_train.pop('Fault')
X_train=mdf_train

y_test=mdf_test.pop('Fault')
X_test=mdf_test

In [5]:
scaler=StandardScaler()
var=list(X_train.columns)
X_train[var]=scaler.fit_transform(X_train[var])
X_train

,Min,Max,Mean,Rms,Var,Std,Power,Peak,P2p,CrestFactor,Skew,Kurtosis,Max_f,Sum_f,Mean_f,Var_f,Peak_f,Skew_f,Kurtosis_f
1000,-0.065247,0.905665,1.881600,0.430708,0.385431,0.430418,0.385815,0.701071,0.490996,0.257905,1.168655,0.064500,0.428462,0.378844,0.385815,0.330375,0.428462,1.168655,0.064500
13101,-0.954744,0.695859,-1.886316,0.753886,0.808843,0.750493,0.813444,0.865544,0.838523,-0.238493,-1.227051,0.048690,0.848364,0.805468,0.813444,0.994740,0.848364,-1.227051,0.048690
9700,1.515187,-1.323692,-1.049747,-1.592548,-1.633339,-1.594922,-1.631592,-1.519323,-1.441608,1.291829,0.984026,0.497566,-1.577877,-1.633821,-1.631592,-1.612002,-1.577877,0.984026,0.497566
10567,1.515187,-1.720293,-0.105202,-1.639010,-1.664096,-1.638296,-1.664556,-1.616389,-1.642051,1.141716,-0.673581,0.319018,-1.610556,-1.666708,-1.664556,-1.626779,-1.610556,-0.673581,0.319018
7747,-0.848756,0.255943,1.517159,0.446423,0.406430,0.446815,0.405937,0.759041,0.562144,0.347056,0.130363,0.127200,0.448091,0.398919,0.405937,0.359442,0.448091,0.130363,0.127200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16304,1.398465,-1.828580,0.179912,-1.724821,-1.723110,-1.723672,-1.723859,-1.499101,-1.637262,2.137173,-0.848205,0.637558,-1.668480,-1.725664,-1.723859,-1.651679,-1.668480,-0.848205,0.637558
79,-0.306739,0.633594,1.334854,0.498628,0.474120,0.499279,0.473277,0.430094,0.476630,-0.546766,0.702786,-0.014881,0.515624,0.466101,0.473277,0.460939,0.515624,0.702786,-0.014881
12119,-0.748134,0.377766,-0.447445,0.520529,0.502207,0.520877,0.501753,0.657930,0.572405,-0.087984,-0.324956,-0.552034,0.543938,0.494509,0.501753,0.504179,0.543938,-0.324956,-0.552034
14147,-0.923887,0.605169,-0.881777,0.545009,0.533105,0.544521,0.533740,0.834537,0.776953,0.244058,0.239269,0.012177,0.574130,0.526421,0.533740,0.550736,0.574130,0.239269,0.012177


In [6]:
X_test[var]=scaler.transform(X_test[var])
X_test

,Min,Max,Mean,Rms,Var,Std,Power,Peak,P2p,CrestFactor,Skew,Kurtosis,Max_f,Sum_f,Mean_f,Var_f,Peak_f,Skew_f,Kurtosis_f
17390,2.089402,-1.863773,0.811618,-1.798190,-1.772133,-1.796931,-1.772940,-2.057232,-2.007365,0.098013,-0.003262,0.349316,-1.715738,-1.774837,-1.772940,-1.670660,-1.715738,-0.003262,0.349316
2000,1.884134,-1.777143,1.161451,-1.713884,-1.715909,-1.713095,-1.716414,-1.970951,-1.858913,0.031325,-1.782152,0.475180,-1.660250,-1.718235,-1.716414,-1.648252,-1.660250,-1.782152,0.475180
5863,-0.913154,0.689091,1.040772,0.666217,0.695863,0.667159,0.694571,0.823752,0.813895,-0.104195,0.595377,-0.159249,0.737242,0.688495,0.694571,0.808914,0.737242,0.595377,-0.159249
6706,-0.533474,0.574036,0.242516,0.610642,0.621745,0.611701,0.620321,0.442227,0.562144,-0.804742,1.153728,-0.044574,0.661358,0.612799,0.620321,0.687830,0.661358,1.153728,-0.044574
1981,2.037079,-1.441454,1.215412,-1.708543,-1.712333,-1.707859,-1.712767,-1.636611,-1.767243,1.445830,1.328378,0.971308,-1.656672,-1.714596,-1.712767,-1.646751,-1.656672,1.328378,0.971308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19941,-0.946694,0.592986,-0.566637,0.671830,0.702364,0.671993,0.702117,0.857455,0.782426,-0.044922,0.398840,-0.237555,0.742012,0.695215,0.702117,0.817361,0.742012,0.398840,-0.237555
11009,-0.666295,0.408899,1.238533,0.570377,0.568077,0.571140,0.567062,0.575693,0.546409,-0.404809,-1.003745,-0.246918,0.605833,0.559665,0.567062,0.600115,0.605833,-1.003745,-0.246918
16995,1.583610,-1.253306,0.372320,-1.666807,-1.683121,-1.665501,-1.683990,-1.449219,-1.440924,2.004594,1.417874,0.570233,-1.630402,-1.686096,-1.683990,-1.635487,-1.630402,1.417874,0.570233
290,-0.297348,0.479285,1.594366,0.490156,0.462644,0.490426,0.462297,0.276405,0.393853,-0.874700,0.292355,-0.051760,0.484694,0.464925,0.462297,0.406951,0.484694,0.292355,-0.051760


In [7]:
xgb=XGBClassifier()
params = {
        'n_estimators' : [100, 200, 500, 750], # no of trees 
        'learning_rate' : [0.01, 0.02, 0.05, 0.1, 0.25],  # eta
        'min_child_weight': [1, 5, 7, 10],
        'gamma': [0.1, 0.5, 1, 1.5, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5, 10, 12]
        }

folds = 3

param_comb = 10 	#It will randomly pick any 1000 combinations of the above hyperparameters

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='accuracy', n_jobs=-1, cv=3, verbose=1, random_state=42)
random_search.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, gamma=None,
                                           gpu_id=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None, max_bin=None,...
                                           predictor=None, random_state=None,
                                           reg_alpha=None, r

In [8]:
xgb_best=random_search.best_estimator_

y_pred_train_xgb_best=xgb_best.predict(X_train)
y_pred_test_xgb_best=xgb_best.predict(X_test)

In [9]:
confusion_matrix(y_train,y_pred_train_xgb_best)

array([[5939,   78,  121],
       [  95, 4918,  121],
       [ 207,  114, 5815]], dtype=int64)

In [10]:
confusion_matrix(y_test,y_pred_test_xgb_best)

array([[1428,   48,   66],
       [  67, 1121,   78],
       [  96,   60, 1388]], dtype=int64)

In [11]:
print(classification_report(y_test,y_pred_test_xgb_best, digits=3))

              precision    recall  f1-score   support

           0      0.898     0.926     0.912      1542
           1      0.912     0.885     0.899      1266
           2      0.906     0.899     0.902      1544

    accuracy                          0.905      4352
   macro avg      0.905     0.903     0.904      4352
weighted avg      0.905     0.905     0.905      4352



In [12]:
accuracy_score(y_train,y_pred_train_xgb_best)

0.9577205882352942

In [13]:
accuracy_score(y_test,y_pred_test_xgb_best)

0.9046415441176471

In [ ]:
import pickle
with open('xgb_model_std_scaled','wb') as f:
    pickle.dump(xgb_best,f)